In [1]:
import os
import cv2
import re
import numpy as np

# Set image size and path
IMG_SIZE = 128
dataset_path = 'C:/Users/Shweta/OneDrive/Desktop/99'  # Update to your dataset path

# Update load_data function to handle files without age information
def load_data(path):
    images = []
    ages = []
    for subdir, _, files in os.walk(path):
        for file in files:
            if file.endswith('.jpg'):
                img_path = os.path.join(subdir, file)
                # Load and preprocess image
                image = cv2.imread(img_path)
                if image is None:
                    continue
                image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
                image = image / 255.0  # Normalize to [0,1]
                images.append(image)
                
                # Extract age using a regular expression
                match = re.search(r'_(\d+)\.jpg', file)
                if match:
                    age = int(match.group(1))
                    ages.append(age)
                else:
                    # If age not found, skip this file
                    print(f"Skipping file with unexpected format: {file}")
                    images.pop()  # Remove the last added image if age not found
    
    # Convert lists to numpy arrays
    images = np.array(images, dtype='float32')
    ages = np.array(ages, dtype='float32')
    return images, ages

# Load data
X, y = load_data(dataset_path)


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2
BATCH_SIZE = 32
# Split into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator()

# Build the CNN model
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1)  # Output layer for age prediction
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

model = build_model()
model.summary()

# Train the model
EPOCHS = 20

history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    validation_data=val_datagen.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    validation_steps=len(X_val) // BATCH_SIZE,
    epochs=EPOCHS
)

# Evaluate the model
loss, mae = model.evaluate(X_val, y_val)
print(f"Validation Mean Absolute Error: {mae:.2f} years")

# Save the model
model.save("age_detection_model.h5")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

...

c:\Users\Shweta\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


106/106 ━━━━━━━━━━━━━━━━━━━━ 43s 377ms/step - loss: 1234152.1250 - mae: 876.7705 - val_loss: 232748.0312 - val_mae: 455.8935
Epoch 2/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 49756.6328 - mae: 171.1729 - val_loss: 258390.8906 - val_mae: 492.9537
Epoch 3/20


c:\Users\Shweta\anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


106/106 ━━━━━━━━━━━━━━━━━━━━ 47s 432ms/step - loss: 90215.9609 - mae: 238.9321 - val_loss: 214305.5781 - val_mae: 445.1790
Epoch 4/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - loss: 97258.6719 - mae: 231.0190 - val_loss: 270621.4688 - val_mae: 494.3406
Epoch 5/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 43s 400ms/step - loss: 79459.7109 - mae: 224.0572 - val_loss: 377455.1875 - val_mae: 604.8687
Epoch 6/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - loss: 93936.6797 - mae: 249.9925 - val_loss: 272700.0625 - val_mae: 509.9067
Epoch 7/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 43s 395ms/step - loss: 73069.9453 - mae: 216.7124 - val_loss: 440950.3750 - val_mae: 655.7880
Epoch 8/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 130551.0391 - mae: 314.1664 - val_loss: 394257.3125 - val_mae: 622.2703
Epoch 9/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 61s 568ms/step - loss: 85076.4922 - mae: 234.6657 - val_loss: 228584.5938 - val_mae: 469.2613
Epoch 10/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - loss: 103300.4922 - 

Validation Mean Absolute Error: 502.35 years
